In [6]:
import requests
import re
import fake_useragent
from bs4 import BeautifulSoup

from os import makedirs

## Константы

In [13]:
STATUS_OK = 200
USER_AGENT = fake_useragent.UserAgent()
MAIN_URL = f"https://www.kinopoisk.ru/"

## Вспомогательный функции

In [14]:
def get_page_text(
        url: str, 
        user_agent: fake_useragent.UserAgent, 
        error_log=True
    ) -> str:
    headers = {"user_agent": user_agent.random}
    response = requests.get(
        url=url,
        headers=headers,
        allow_redirects=True
    )

    if response.status_code == STATUS_OK:
        return response.text
    elif error_log:
        print(f"Error, response code: {response.status_code}\n URL: {url}")

    return None


def clear_link(page_link: str) -> str:
    page_link = re.sub(r"^\/", '', page_link)
    page_link = re.sub(r"[\#\?].+", '', page_link)
    return page_link

def get_page_links(soup: BeautifulSoup, main_url: str) -> set:
    links = set()
    for row in soup.find_all('a'):
        if not row.has_attr("href"):
            continue

        page_link = row.attrs["href"]

        if re.search("^(http)", page_link):
            if main_url not in page_link or main_url == page_link:
                continue
        
        page_link = clear_link(page_link=page_link)

        if len(page_link) == 0:
            continue

        links.add(main_url + page_link)

    return links

In [15]:
page_text = get_page_text(MAIN_URL, USER_AGENT)
soup = BeautifulSoup(page_text)

all_links = get_page_links(soup=soup, main_url=MAIN_URL)
process_link = all_links

while True:
    new_links = set()

    for link in process_link:
        page_text = get_page_text(url=link, user_agent=USER_AGENT)

        soup = BeautifulSoup(page_text)
        links = get_page_links(soup=soup, main_url=MAIN_URL)
        links = links.difference(all_links, new_links)

        new_links = new_links.union(links)

    if len(new_links) == 0:
        break

    all_links = all_links.union(new_links)
    process_links = new_links

In [ ]:
"../repositories/page_texts/"
"../repositories/row_pages/"